## Подготовка данных для занятия

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 8.2MB 11.2MB/s 


In [4]:
import re

import pandas as pd
import pymorphy2


from google.colab import drive
from pathlib import Path
drive.mount('/content/drive/')
path = Path('/content/drive/')

Mounted at /content/drive/


In [5]:
text_df = pd.read_csv("/content/drive/MyDrive/Python/19_data/data-2/content_description.csv", sep='\t')
text_df.head()

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...


In [6]:
print(text_df.loc[0, 'description'])

Лучший подарок, который только можно было придумать для всех поклонников культового сериала  – рождественский спецвыпуск «Шерлока», в котором абсолютно все будет особенным: и старинные костюмы, и дух викторианской Англии, и, конечно же, покрытая  мраком и завесой мистики тайна, которую предстоит разгадать гениальному сыщику. Кроме того, создатели сериала обещали оставить для самых внимательных поклонников «Шерлока» подсказки, связанные с содержанием четвертого сезона сериала.  Впервые герои современного сериала предстанут в своих «книжных» образах – Холмс с трубкой и в охотничьей шляпе, а Ватсон – с усами и в котелке. Действие перенесено в 1895 год. В центре сюжета загадочная история о призраке невесты, который наводит ужас на весь город. К Шерлоку за помощью обращается инспектор Лестрейд, который расследует дело об убийстве Томаса Риколетти. Жена Риколетти покончила с собой. Спустя некоторое время мистер Риколетти увидел супругу живой и в свадебном платье, после чего покойная Эмилия в

In [7]:
# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).tail(10)

,doc_id,word,dummy
45,0,на,1
44,0,мрак,1
42,0,мистика,1
41,0,мистер,1
40,0,лестрейд,1
39,0,культовый,1
38,0,кроме,1
36,0,котелок,1
35,0,костюм,1
114,0,этот,1


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процент документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [8]:
doc_frequency = word_count_df.groupby('word')['doc_id'].count().reset_index(name='doc_freq').sort_values(by='doc_freq', ascending=False)
doc_frequency.doc_freq /= float(doc_count)
doc_frequency

,word,doc_freq
171,и,1.000
41,в,1.000
269,на,0.875
468,с,0.875
323,он,0.750
...,...,...
235,лишаться,0.125
236,лос,0.125
237,любить,0.125
238,любой,0.125


In [9]:
doc_frequency = word_count_df.groupby('word')['doc_id'].count().reset_index(name='doc_freq').sort_values(by='doc_freq', ascending=False)
doc_frequency.doc_freq /= float(doc_count)
doc_frequency

,word,doc_freq
171,и,1.000
41,в,1.000
269,на,0.875
468,с,0.875
323,он,0.750
...,...,...
235,лишаться,0.125
236,лос,0.125
237,любить,0.125
238,любой,0.125


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [11]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
# регулярка - её нужно поправить
reg_expr = r'[А-Я]\w\w*.[А-Я]\w\w\w\w*'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Стивена Фрая
Джоном Дженксом
Теда Уоллеса


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [12]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])

# -- ВАШ КОД ЗДЕСЬ --
genres_df = genres_df.merge(word_count_df, left_on="character_word", right_on='word').drop(['word', 'character_word', 'dummy'], axis=1)
genres_df = genres_df.drop_duplicates()
genres_df

,genre,doc_id
0,комедия,3
1,комедия,4
3,мелодрама,2
5,сказка,1
6,детектив,0
9,триллер,0
10,триллер,6
